# SELECT within SELECT

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app04")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@466d3665

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@6856ae5e

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val world = hiveCxt.table("sqlzoo.world")

world: DataFrame = [name: string, continent: string ... 6 more fields]

## 1. Bigger than Russia

**List each country name where the population is larger than that of 'Russia'.**

```
world(name, continent, area, population, gdp)
```

In [5]:
(world.filter($"population" > world.filter($"name"==="Russia")
              .select("population")
              .collect()(0)(0))
 .select($"name").showHTML())

name
Bangladesh
Brazil
China
India
Indonesia
Nigeria
Pakistan
United States


## 2. Richer than UK

**Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.**

> _Per Capita GDP_   
> The per capita GDP is the gdp/population

In [6]:
val a = world.withColumn("pcgdp", $"gdp"/$"population")
(a.filter(($"pcgdp" > a.filter($"name"==="United Kingdom")
          .select("pcgdp").head()(0)) && 
         ($"continent"==="Europe"))
 .select($"name").showHTML())

name
Andorra
Austria
Belgium
Denmark
Finland
Germany
Iceland
Ireland
Liechtenstein
Luxembourg


a: DataFrame = [name: string, continent: string ... 7 more fields]

## 3. Neighbours of Argentina and Australia


List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

In [7]:
val bm = (world.filter($"name".isin(List("Argentina", "Australia"): _*))
          .select($"continent").collect()
          .map(_(0)).toList)

world.filter($"continent".isin(bm:_*)).select($"name", $"continent").showHTML()

name,continent
Argentina,South America
Australia,Oceania
Bolivia,South America
Brazil,South America
Chile,South America
Colombia,South America
Ecuador,South America
Fiji,Oceania
Guyana,South America
Kiribati,Oceania


bm: List[Any] = List("South America", "Oceania")

## 4. Between Canada and Poland

Which country has a population that is more than Canada but less than Poland? Show the name and the population.

In [8]:
val popl_can = world.filter($"name"==="Canada").select($"population").head()(0)
val popl_pol = world.filter($"name"==="Poland").select($"population").head()(0)
(world.filter(($"population" > popl_can) &&
              ($"population" < popl_pol))
 .select($"name", $"population").showHTML())

name,population


popl_can: Any = 3.8007166E7
popl_pol: Any = 3.8379E7

## 5. Percentages of Germany

Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

**Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.**

The format should be Name, Percentage for example:

name	| percentage
--------|-----------
Albania	| 3%
Andorra	| 0%
Austria	| 11%
...	| ...

> _Decimal places_   
> You can use the function ROUND to remove the decimal places.

> _Percent symbol %_
> You can use the function CONCAT to add the percentage symbol.

[To get a well rounded view of the important features of SQL you should move on to the next tutorial concerning aggregates.](https://sqlzoo.net/wiki/SUM_and_COUNT)

To gain an absurdly detailed view of one insignificant feature of the language, read on.

We can use the word `ALL` to allow >= or > or < or <=to act over a list. For example, you can find the largest country in the world, by population with this query:

```sql
SELECT name
  FROM world
 WHERE population >= ALL(SELECT population
                           FROM world
                          WHERE population>0)
```

You need the condition **population>0** in the sub-query as some countries have **null** for population.

In [9]:
val bm = world.filter($"name"==="Germany").select("population").head()(0)
(world.filter($"continent"==="Europe")
    .withColumn("pct", format_string("%d%%", floor(round(lit(100) * $"population"/bm, 0))))
    .select($"name", $"pct").showHTML())

name,pct
Albania,3%
Andorra,0%
Austria,11%
Belarus,11%
Belgium,14%
Bosnia and Herzegovina,4%
Bulgaria,8%
Croatia,5%
Czech Republic,13%
Denmark,7%


bm: Any = 8.31493E7

## 6. Bigger than every country in Europe

Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

We can refer to values in the outer SELECT within the inner SELECT. We can name the tables so that we can tell the difference between the inner and outer versions.

In [10]:
(world.filter($"gdp" > world.filter(
    $"continent"==="Europe").agg(max($"gdp")).head()(0))
 .select($"name").showHTML())

name
China
Japan
United States


## 7. Largest in each continent

**Find the largest country (by area) in each continent, show the continent, the name and the area:**

```sql
SELECT continent, name, population FROM world x
  WHERE population >= ALL
    (SELECT population FROM world y
        WHERE y.continent=x.continent
          AND population>0)
```

> __The above example is known as a correlated or synchronized sub-query.__   
> 
> Using correlated subqueries
> A correlated subquery works like a nested loop: the subquery only has access to rows related to a single > record at a time in the outer query. The technique relies on table aliases to identify two different uses of the same table, one in the outer query and the other in the subquery.
> 
> One way to interpret the line in the **WHERE** clause that references the two table is _“… where the correlated values are the same”._
> 
> In the example provided, you would say _“select the country details from world where the population is greater than or equal to the population of all countries where the continent is the same”._

In [11]:
val max_area = (world.groupBy($"continent")
            .agg(max($"area").as("area")))
(world.join(max_area, Seq("continent", "area"), joinType="inner")
     .select("continent", "name", "area")
     .showHTML())

continent,name,area
Africa,Algeria,2381741.0
Oceania,Australia,7692024.0
South America,Brazil,8515767.0
North America,Canada,9984670.0
Asia,China,9596961.0
Caribbean,Cuba,109884.0
Europe,Kazakhstan,2724900.0
"Federated States of,Oceania",Micronesia,702.0
Eurasia,Russia,1.7125242E7


max_area: DataFrame = [continent: string, area: double]

## 8. First country of each continent (alphabetically)

**List each continent and the name of the country that comes first alphabetically.**

In [12]:
(world.groupBy("continent")
     .agg(min("name").as("name"))
     .showHTML())

continent,name
"Federated States of,Oceania",Micronesia
Africa,Algeria
Asia,Afghanistan
Caribbean,Antigua and Barbuda
Eurasia,Armenia
Europe,Albania
North America,Belize
Oceania,Australia
South America,Argentina


## 9. Difficult Questions That Utilize Techniques Not Covered In Prior Sections

**Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.**

In [13]:
val bm = (world.filter($"population" > 2.5e7)
          .select("continent").distinct()
          .collect().map(_(0)).toList)
(world.filter(! $"continent".isin(bm: _*))
 .select("name", "continent", "population")
 .showHTML())

name,continent,population
Antigua and Barbuda,Caribbean,96453.0
Bahamas,Caribbean,385340.0
Barbados,Caribbean,287025.0
Cuba,Caribbean,1.1209628E7
Dominica,Caribbean,71808.0
Dominican Republic,Caribbean,1.035832E7
Grenada,Caribbean,112003.0
Haiti,Caribbean,1.1577779E7
Jamaica,Caribbean,2726667.0
Micronesia,"Federated States of,Oceania",101351.0


bm: List[Any] = List(
  "Europe",
  "Eurasia",
  "Africa",
  "North America",
  "South America",
  "Oceania",
  "Asia"
)

## 10.

**Some countries have populations more than three times that of any of their neighbours (in the same continent). Give the countries and continents.**

In [14]:
// how many neighbors are there for each continent
val bm = (world.groupBy("continent")
          .agg(count($"name").as("nbhrs"))
          .withColumn("nbhrs", $"nbhrs"-lit(1)))
// compare with tripple population
val a = world.join(world
        .withColumn("popl3", lit(3) * $"population")
        .select("continent", "name", "popl3")
        .withColumnRenamed("name", "name_y"),
    "continent", joinType="outer")
val b = (a.filter($"population" > $"popl3")
         .select("continent", "name_y", "name")
         .groupBy($"continent", $"name")
         .agg(count("name").as("cnt"))
         .join(bm, "continent", joinType="left"))
b.filter($"cnt"===$"nbhrs").select("name", "continent").showHTML()

name,continent
Russia,Eurasia
Brazil,South America


bm: DataFrame = [continent: string, nbhrs: bigint]
a: DataFrame = [continent: string, name: string ... 8 more fields]
b: DataFrame = [continent: string, name: string ... 2 more fields]

In [15]:
spark.stop()